In [36]:
from osman import Osman, OsmanConfig
from opensearchpy import OpenSearch
import pandas as pd
import re

In [37]:
OS_HOST_URL="http://localhost:9200"
TEMPLATE_NAME = 'semantic_search'
INDEX_NAME = "bible-nlp"
ID_KEY = "id"

In [38]:
host = 'localhost'
port = 9200

# Connect to OpenSearch
client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)

In [39]:
os_man = Osman(OsmanConfig(host_url=OS_HOST_URL))

In [40]:
# Open the file and read its contents
with open('bible.txt', 'r') as file:
    bible_text = file.read()

In [41]:
# Split the text using regular expressions
verses = re.split(r'(\d+:\d+)', bible_text)

In [42]:
bible_dict = {}
# Iterate in steps of 2 starting from 1, since 0 is likely an empty string before the first verse
for i in range(1, len(verses), 2):
    verse_number = verses[i]
    verse_text = verses[i + 1].strip()  # Use strip() to remove any leading/trailing whitespace
    bible_dict[verse_number] = verse_text


In [43]:
def generate_id(verse_id):
    chapter, verse = verse_id.split(':')
    return int(chapter) * 1000 + int(verse)  # Convert chapter:verse into a unique integer ID

documents = [
    {"verse_id": verse_id, "content": content, "id": generate_id(verse_id)}
    for verse_id, content in bible_dict.items()
]


In [44]:
# Insert data into OpenSearch index
response = os_man.add_data_to_index(
    index_name=INDEX_NAME,
    documents=documents,
    id_key=ID_KEY,
    refresh=True  # Set to True for immediate availability, but be cautious in production
)


In [45]:
# Extracting the data from the response
hits = response['hits']['hits']
data = [
    {
        "Index": hit['_index'],
        "Score": hit['_score'],
        "Verse ID": hit['_source']['verse_id'],
        "Content": hit['_source']['content'].replace('\n', ' ')  # Replacing newline characters for better display
    }
    for hit in hits
]

# Creating a DataFrame
df = pd.DataFrame(data)

KeyError: 'hits'

In [ ]:
df

,Index,Score,Verse ID,Content
0,bible,7.438810,15:23,But every man in his own order: Christ the fir...
1,bible,6.188467,3:27,For as many of you as have been baptized into ...
2,bible,5.771050,16:23,The grace of our Lord Jesus Christ be with you.
3,bible,5.536330,26:68,"Saying, Prophesy unto us, thou Christ, Who is ..."
4,bible,5.319957,16:22,"If any man love not the Lord Jesus Christ, let..."
5,bible,5.217992,15:57,"But thanks be to God, which giveth us the vict..."
6,bible,5.217992,15:22,"For as in Adam all die, even so in Christ shal..."
7,bible,5.091649,15:16,"For if the dead rise not, then is not Christ r..."
8,bible,5.091649,15:18,Then they also which are fallen asleep in Chri...
9,bible,4.942670,5:32,This is a great mystery: but I speak concernin...
